In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import warnings
from pandas import Series
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from learntools.core import binder
binder.bind(globals())
from learntools.machine_learning.ex3 import *
import seaborn as sns

In [ ]:
import datetime
df_cases1 = pd.read_csv('/kaggle/input/us-counties-covid-19-dataset/us-counties.csv')
df_cases = df_cases1.query("county == 'Marion'") 
df_cases = df_cases.query("state == 'Oregon'")

d1 = datetime.date(2020, 3, 8)
df_cases['Date_2'] = pd.to_datetime(df_cases['date']).dt.date
df_cases['days_from_outbreak'] = df_cases['Date_2'] - d1
df_cases['days'] = (df_cases['days_from_outbreak'] / np.timedelta64(1, 'D')).astype(int)
df_cases.tail()

In [ ]:
#df_marion_polk = df_cases1[(df_cases1.county == 'Marion') | (df_cases1.county == 'Polk')]
#df_marion_polk = df_marion_polk.query("state == 'Oregon'")
#df_marion_polk.sort_values(by=['date'])
#df_marion_polk.tail()

df_polk = df_cases1[(df_cases1.county == 'Polk')]
df_polk = df_polk.query("state == 'Oregon'")
df_polk.sort_values(by=['date'])


df_polk['Date_2'] = pd.to_datetime(df_polk['date']).dt.date
df_polk['days_from_outbreak'] = df_polk['Date_2'] - d1
df_polk['days'] = (df_polk['days_from_outbreak'] / np.timedelta64(1, 'D')).astype(int)

df_polk.set_index("days", inplace = True)
df_polk['new_cases'] = df_polk['cases'] - df_polk['cases'].shift(1)

df_polk.tail()

In [ ]:
df_cases.set_index("days", inplace = True)


df_cases['new_cases'] = df_cases['cases'] - df_cases['cases'].shift(1)

#max(a)
df_cases.tail()

In [ ]:
df_mar_polk = df_cases
df_mar_polk['combined'] = df_cases['new_cases'] + df_polk['new_cases']
#df_mar_polk.dropna

#corrects errant 0 value for this day
df_mar_polk.iat[160,9] = 39

df_mar_polk['ewma7'] = pd.Series.ewm(df_mar_polk['combined'], span=7).mean()
df_mar_polk['ewma14'] = pd.Series.ewm(df_mar_polk['combined'], span=14).mean()

df_mar_polk.tail(10)

In [ ]:
df_mobility = pd.read_csv('/kaggle/input/covid19-mobility-data/Global_Mobility_Report.csv', low_memory=False)

df_mobility = df_mobility[(df_mobility.census_fips_code == 41047) | (df_mobility.census_fips_code == 41053)]

df_mobility.drop(['country_region_code', 'country_region', 'sub_region_1', 'sub_region_2', 'iso_3166_2_code', 'census_fips_code', 'metro_area'], axis=1, inplace=True)

#df_mobility.groupby(['date']).mean()
df_mobility['date'] = pd.to_datetime(df_mobility['date']).dt.date
df_mobility.set_index('date', inplace = True)
df_mobility.groupby(['date']).mean().sort_values(by=['date'])

#df_mobility.head()

In [ ]:

df_mobility.head()

In [ ]:
df2 = df_mar_polk.merge(df_mobility, left_on='Date_2', right_on='date')
df2.interpolate(inplace = True)
df2.dropna(inplace=True)
df2.tail()

In [ ]:
df2.drop(columns=['fips'], inplace = True)
df2.corr()

In [ ]:
df_cases['ewma7'] = pd.Series.ewm(df_cases['new_cases'], span=7).mean()
df_cases['ewma14'] = pd.Series.ewm(df_cases['new_cases'], span=14).mean()

ax = sns.lineplot(x=df_cases.Date_2, y='ewma7', data=df_cases)
ax.set(xlabel='\n Date', ylabel= '\n 7 Day Exponentially Weighted Average')
fig = plt.gcf()
fig.set_size_inches( 24, 10)

for item in ax.get_xticklabels():
    item.set_rotation(0)

In [ ]:
# import graph objects as "go"
import plotly.graph_objs as go
import plotly.io as pio


# Creating trace1
trace1 = go.Scatter(
                    x = df_cases.Date_2,
                    y = df_cases.ewma7,
                    mode = "lines",
                    name = "EWMA of New COVID-19 Cases",
                    marker = dict(color = 'rgba(16, 112, 2, 0.8)'),
                    text= df_cases.new_cases)

data = [trace1]
layout = dict(title = '7 Day Exponentially Weighted Moving Average for COVID-19 Cases in Marion County, Oregon',
              xaxis= dict(title= 'Date',zeroline= False)
             )
fig = dict(data = data, layout = layout)
#iplot(fig)
pio.show(fig)

In [ ]:
trace1 = go.Scatter(
                    x = df_cases.Date_2,
                    y = df_cases.ewma14,
                    mode = "lines",
                    name = "EWMA of New COVID-19 Cases",
                    marker = dict(color = 'rgba(16, 112, 2, 0.8)'),
                    text= df_cases.new_cases)

data = [trace1]
layout = dict(title = '14 Day Exponentially Weighted Moving Average for COVID-19 Cases in Marion County, Oregon',
              xaxis= dict(title= 'Date',zeroline= False)
             )
fig = dict(data = data, layout = layout)
#iplot(fig)
pio.show(fig)

In [ ]:
trace1 = go.Scatter(
                    x = df_mar_polk.Date_2,
                    y = df_mar_polk.ewma14,
                    mode = "lines",
                    name = "EWMA of New COVID-19 Cases",
                    marker = dict(color = 'rgba(16, 112, 2, 0.8)'),
                    text= df_mar_polk.combined)

data = [trace1]
layout = dict(title = '14 Day Exponentially Weighted Moving Average for COVID-19 Cases in Marion & Polk County, Oregon',
              xaxis= dict(title= 'Date',zeroline= False)
             )
fig = dict(data = data, layout = layout)
#iplot(fig)
pio.show(fig)

In [ ]:
y = df2.combined
df2['days'] = (df2['days_from_outbreak'] / np.timedelta64(1, 'D')).astype(int)
df2_features = ['days','retail_and_recreation_percent_change_from_baseline','grocery_and_pharmacy_percent_change_from_baseline','parks_percent_change_from_baseline','transit_stations_percent_change_from_baseline', 'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline']
X = df2[df2_features]
X.describe()

In [ ]:
from sklearn.tree import DecisionTreeRegressor

COVID_model = DecisionTreeRegressor(random_state=1)

COVID_model.fit(X, y)

print("Making predictions for the following 5 days:")
print(X.head())
print("The predictions are")
print(COVID_model.predict(X.head()))

In [ ]:
from sklearn.metrics import mean_absolute_error

predicted_COVID = COVID_model.predict(X)
mean_absolute_error(y, predicted_COVID)

In [ ]:
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)

COVID_model = DecisionTreeRegressor()

COVID_model.fit(train_X, train_y)


val_predictions = COVID_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

In [ ]:
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
COV_preds = forest_model.predict(val_X)
print(mean_absolute_error(val_y, COV_preds))

In [ ]:
import fbprophet
# Prophet requires columns ds (Date) and y (value)
df3 = df2.rename(columns={'date': 'ds', 'combined': 'y'})

# Make the prophet model and fit on the data
df3_prophet = fbprophet.Prophet(changepoint_prior_scale=0.5)
df3_prophet.fit(df3)

# Make a future dataframe for 2 years
df3_forecast = df3_prophet.make_future_dataframe(periods=30, freq='D')
# Make predictions
df3_forecast = df3_prophet.predict(df3_forecast)

In [ ]:
df3_prophet.plot(df3_forecast, xlabel = 'Date', ylabel = 'COVID cases (projected)')
plt.title('COVID');

In [ ]:
# Prophet requires columns ds (Date) and y (value)
df4 = df2.rename(columns={ 'ds' : 'date', 'Date_2': 'ds', 'y' : 'combined', 'ewma14': 'y'})

# Make the prophet model and fit on the data
df4_prophet = fbprophet.Prophet(weekly_seasonality = True, changepoint_prior_scale=.15)
#df4_prophet = fbprophet.Prophet(changepoints=['2020-03-23','2020-05-22','2020-06-19','2020-07-01','2020-07-15'])

df4_prophet.fit(df4)

# Make a future dataframe for 2 years
df4_forecast = df4_prophet.make_future_dataframe(periods=30, freq='D')



# Make predictions
df4_forecast = df4_prophet.predict(df4_forecast)

#df4_prophet.changepoints

In [ ]:

df4_prophet.plot(df4_forecast, xlabel = 'Date', ylabel = 'COVID cases (with future projected cases)')

plt.title('14 Day Exponentially Weighted Moving Average for COVID-19 Cases in Marion & Polk County, Oregon');

# for changepoint in df4_prophet.changepoints:
#     plt.axvline(changepoint,ls='--', lw=1)

In [ ]:
deltas = df4_prophet.params['delta'].mean(0)
fig = plt.figure(facecolor='w')
ax = fig.add_subplot(111)
ax.bar(range(len(deltas)), deltas)
ax.grid(True, which='major', c='gray', ls='-', lw=1, alpha=0.2)
ax.set_ylabel('Rate change')
ax.set_xlabel('Potential changepoint')
fig.tight_layout()

In [ ]:
from fbprophet.plot import plot_plotly
import plotly.offline as py
py.init_notebook_mode()

fig = plot_plotly(df4_prophet, df4_forecast)  # This returns a plotly Figure
py.iplot(fig)

In [ ]:
df4_prophet.plot_components(df4_forecast)